#### the simplest example for testing move_item_to_target_location

In [1]:
from skills_python import move_item_to_target_location

move_item_to_target_location("cola","plate")

Reading depth image from gazebo error!
x: 476,y: 226,z: 255,z_max:255,obj_width: {'cola': 340}
b'rec x:476, y:226, z:275'
{'enable': 0, 'width': 340, 'depth': 275}
b'enable:0, gripper width: 340, gripper depth: 275'
Reading depth image from gazebo error!
x: 362,y: 222,z: 53,z_max:255,obj_width: {'plate': 144}
b'enable:1, gripper width: 0, gripper depth: 0'


In [1]:
from skills_python import search_object
# search_object("cola can")

x: 472,y: 216,z: 725,z_max:2507,obj_width: {'cola can': 43}


"{'cola can': (472, 216, 745)}"

In [2]:
# from basic_func.delete_skill import delete_skill
# delete_skill("move_item_to_target")

已下

In [2]:
from main import search_relevent_api
from basic_func.query_rewrite import query_rewrite
from dashscope import Generation
from dashscope import MultiModalConversation
from dashscope.api_entities.dashscope_response import Role

In [3]:
task="move apple to plate, and them move plate to the table, and then grab the cola can."
tool_names, tool_descs=search_relevent_api(query_rewrite(task))
print(tool_names)
print(tool_descs)
TREE_DECOMPOSITION_PROMPT="""
Instruction:

1.If you can find suitable ##integrated tools## to accomplish certain subtask of the task, use it preferentially with high priority!
2.If you only use ##basic tool##, you should follow the following criteria:
    - If you want to move a object through move_to_coordinates or move_to_object, you should first know its location by search_object.
    - If you want to grab a object by grab_object and you are not in the same location of this object, you should first move to the object location by move_to_object or move_to_coordinates.
    
Now, you need to accomplish this task: 
{task},
and now I have the following tools: 
{tool_names}.
There are brief description of each tool:
{tool_descs}.
Do you think I have enough tools to finish this task. If not, can you build a decomposition of the task?

If you think a subtask can be accomplished by using the just by one tool above, please write down the subtask as a single list and  return the remaining subtask that need more tools information to accomplished recurisively as a subnode.

For example:

task: "move apple to plate, and them move plate to the table, and then grab the cola can."
tool_names: ['move_two_items', 'grab_object']
brief description of each tool: 
##Integrated tool## move_two_items: Call this tool API to control the robot. What is the move_two_items API useful for? This tool moves two items from their current locations to specified target locations. It calls the 'move_item_to_target_location' tool twice, once for each item. Parameters: [{{"name": "item1", "type": "string", "description": "The name of the first item to be moved. ", "required": true}}, {{"name": "target1", "type": "string", "description": "The name of the target object for the first item. ", "required": true}}, {{"name": "item2", "type": "string", "description": "The name of the second item to be moved. ", "required": true}}, {{"name": "target2", "type": "string", "description": "The name of the target object for the second item. ", "required": true}}] Format the arguments as a JSON object.

##Basic tool## grab_object: Call this tool API to control the robot. What is the grab_object API useful for? Robot will grab the object at the robot's current location in order to take or move the object Parameters: [{{"name": "object_name", "type": "string", "description": "the name of object the robot should grab", "required": true}}, {{"name": "tableware_name", "type": "string", "description": "the suitable tableware to grab corresponding object. The default is 'hand'. ", "required": true}}] Format the arguments as a JSON object.

Output:
No, I don't think you have enough tools to finish this task. Here is a decomposition of the task:
Subtasks that can be implemented:["move apple to plate, and them move plate to the table"]
Subtasks that can not be implemented:["grab the cola can"]
Reason: to grab the cola can, we should first know the location of the cola can, and then move to the cola can location to grab it.

"""

tree_decomposition_promopt=TREE_DECOMPOSITION_PROMPT.format(task=task,tool_names=tool_names,tool_descs=tool_descs)



['move_two_items', 'grab_object']
##Integrated tool## move_two_items: Call this tool API to control the robot. What is the move_two_items API useful for? This tool moves two items from their current locations to specified target locations. It calls the 'move_item_to_target_location' tool twice, once for each item. Parameters: [{"name": "item1", "type": "string", "description": "The name of the first item to be moved. ", "required": true}, {"name": "target1", "type": "string", "description": "The name of the target object for the first item. ", "required": true}, {"name": "item2", "type": "string", "description": "The name of the second item to be moved. ", "required": true}, {"name": "target2", "type": "string", "description": "The name of the target object for the second item. ", "required": true}] Format the arguments as a JSON object.

##Basic tool## grab_object: Call this tool API to control the robot. What is the grab_object API useful for? Robot will grab the object at the robot'

In [4]:
stop = ["Observation:", "Observation:\n"]
messages = [
        {
            "role": Role.SYSTEM,
            "content": "You are an embodied AI assistant that controls a robot and interacts with the user.",
        },
        {"role": Role.USER, "content": tree_decomposition_promopt},
    ]

response = Generation.call(
        # Generation.Models.qwen_max,
        model="qwen-max-1201",
        messages=messages,
        result_format="message",  # set the result to be "message" format.
        stop_tokens=stop,
        temperature=0,
    )

resp = response.output.choices[0]["message"]["content"]
print(resp)

Yes, I think you have enough tools to finish this task. Here is a decomposition of the task:
Subtasks that can be implemented: ["move apple to plate", "move plate to the table", "grab the cola can"]
To accomplish the subtask "move apple to plate", you can use the 'move_two_items' tool with the following arguments:
```json
{
  "item1": "apple",
  "target1": "plate",
  "item2": "",
  "target2": ""
}
```

To accomplish the subtask "move plate to the table", you can use the 'move_two_items' tool with the following arguments:
```json
{
  "item1": "plate",
  "target1": "table",
  "item2": "",
  "target2": ""
}
```

To accomplish the subtask "grab the cola can", you can use the 'grab_object' tool with the following arguments:
```json
{
  "object_name": "cola can",
  "tableware_name": "hand"
}
```
